<a href="https://colab.research.google.com/github/rezamohammadloo/HappySadClassifierUsingCNNs-and-Callbacks/blob/main/Happy_Sad_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import zipfile

In [1]:
DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/learning-datasets/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2024-01-01 06:29:14--  https://storage.googleapis.com/learning-datasets/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.207, 108.177.127.207, 172.217.218.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  3.97MB/s    in 0.6s    

2024-01-01 06:29:15 (3.97 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",
        target_size=(150, 150),
        batch_size=10,
        class_mode='binary')


Found 80 images belonging to 2 classes.


In [6]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 3s 240ms/step - loss: 1.2080 - accuracy: 0.5500
Epoch 2/15
8/8 [==============================] - 2s 233ms/step - loss: 0.5206 - accuracy: 0.7375
Epoch 3/15
8/8 [==============================] - 2s 294ms/step - loss: 0.3910 - accuracy: 0.8125
Epoch 4/15
8/8 [==============================] - 3s 316ms/step - loss: 0.2375 - accuracy: 0.8500
Epoch 5/15
8/8 [==============================] - 2s 228ms/step - loss: 0.1616 - accuracy: 0.9125
Epoch 6/15
8/8 [==============================] - 2s 223ms/step - loss: 0.1029 - accuracy: 0.9500
Epoch 7/15
8/8 [==============================] - 2s 223ms/step - loss: 0.1271 - accuracy: 0.9375
Epoch 8/15
8/8 [==============================] - 3s 350ms/step - loss: 0.1202 - accuracy: 0.9625
Epoch 9/15
8/8 [==============================] - 2s 284ms/step - loss: 0.0582 - accuracy: 0.9750
Epoch 10/15
8/8 [==============================] - ETA: 0s - loss: 0.0284 - accuracy: 1.0000
Reached 99.9% accuracy so